In [140]:
# Import libraries
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [141]:
metadata = pd.read_csv(r"D:\كتب الكلية\Graduation_Project\Metadata_Dataset\Udacity_final.csv", encoding="cp1252")
metadata

,Course_Name,Organization,Level,Rating,Link,About,Skills
0,Data Engineer,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-engineer-...,Data Engineering is the foundation for the new...,"Data Modeling, Data Pipelines, Data Lakes, Spa..."
1,Data Scientist,School of Data Science,advanced,4.7,https://www.udacity.com//course/data-scientist...,"Build effective machine learning models, run d...","Machine Learning, Deep Learning, Software Engi..."
2,Data Analyst,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-analyst-n...,"Use Python, SQL, and statistics to uncover ins...","Data Wrangling, Matplotlib, Bootstrapping, Pan..."
3,C++,School of Autonomous Systems,Intermediate,4.6,https://www.udacity.com//course/c-plus-plus-na...,Get hands-on experience by building five real-...,"Data Structures &amp; Algorithms, Memory Manag..."
4,Product Manager,School of Product Management,Beginner,4.7,https://www.udacity.com//course/product-manage...,Envision and execute the development of indust...,"Product Strategy, Product Design, Product Deve..."
...,...,...,...,...,...,...,...
258,Front-End Interview Prep,Career Advancement,Intermediate,NaN,https://www.udacity.com//course/front-end-inte...,Answer front-end technical and behavioral inte...,"Interview questions, Common FE Questions, Whit..."
259,Full-Stack Interview Prep,Career Advancement,Intermediate,NaN,https://www.udacity.com//course/full-stack-int...,Answer common full stack and web security inte...,"Interview practice, Common FS Questions, White..."
260,Data Structures & Algorithms in Swift,Career Advancement,Intermediate,NaN,https://www.udacity.com//course/data-structure...,Review and practice the skills technical inter...,"Interview practice, Swift, Data structures, Ca..."
261,iOS Interview Prep,Career Advancement,Intermediate,NaN,https://www.udacity.com//course/ios-interview-...,Answer iOS and mobile development interview qu...,"Interview practice, Common iOS Questions, Whit..."


In [142]:
metadata['About'] = metadata['About'].fillna("")
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Course_Name   263 non-null    object 
 1   Organization  263 non-null    object 
 2   Level         259 non-null    object 
 3   Rating        55 non-null     float64
 4   Link          263 non-null    object 
 5   About         263 non-null    object 
 6   Skills        248 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.5+ KB


In [143]:
metadata['Skills'] = metadata['Skills'].fillna(" ")
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Course_Name   263 non-null    object 
 1   Organization  263 non-null    object 
 2   Level         259 non-null    object 
 3   Rating        55 non-null     float64
 4   Link          263 non-null    object 
 5   About         263 non-null    object 
 6   Skills        263 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.5+ KB


In [144]:
metadata['text'] = metadata['About'] + " " + metadata['Skills']
metadata['text']

0      Data Engineering is the foundation for the new...
1      Build effective machine learning models, run d...
2      Use Python, SQL, and statistics to uncover ins...
3      Get hands-on experience by building five real-...
4      Envision and execute the development of indust...
                             ...                        
258    Answer front-end technical and behavioral inte...
259    Answer common full stack and web security inte...
260    Review and practice the skills technical inter...
261    Answer iOS and mobile development interview qu...
262    Learn how to tackle interview questions for te...
Name: text, Length: 263, dtype: object

In [145]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text, disable=['ner', 'pos'])
    lemmas = [token.lemma_.lower() for token in doc]
    clean_lemmas = [lemma for lemma in lemmas if lemma.isalpha() and lemma not in STOP_WORDS]
    return ' '.join(clean_lemmas)

In [146]:
metadata['clean_text'] = metadata['text'].apply(preprocess_text)
metadata['clean_text']

0      data engineering foundation new world big data...
1      build effective machine learning model run dat...
2      use python sql statistic uncover insight commu...
3      hand experience build real world project data ...
4      envision execute development industry define p...
                             ...                        
258    answer end technical behavioral interview ques...
259    answer common stack web security interview que...
260    review practice skill technical interviewer ex...
261    answer ios mobile development interview questi...
262    learn tackle interview question technical role...
Name: clean_text, Length: 263, dtype: object

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(max_features = 5000, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(metadata['clean_text'])
cosine_sim = cosine_similarity(tfidf_matrix)

In [148]:
nlp = spacy.load("en_core_web_md")

# Convert each course description into a vector
metadata['spacy_vector'] = metadata['clean_text'].apply(lambda x: nlp(str(x)).vector)

# Stack them into a matrix for similarity search
spacy_matrix = np.vstack(metadata['spacy_vector'].values)

In [149]:
from sklearn.preprocessing import normalize

tfidf_matrix_norm = normalize(tfidf_matrix)  
spacy_matrix_norm = normalize(spacy_matrix)

In [150]:

# Compute similarity separately
tfidf_sim = cosine_similarity(tfidf_matrix_norm, tfidf_matrix_norm)
spacy_sim = cosine_similarity(spacy_matrix_norm, spacy_matrix_norm)

# Combine them (both are (263, 263))
alpha = 0.6
beta = 0.4
hybrid_sim = alpha * tfidf_sim + beta * spacy_sim

similarity_matrix = cosine_similarity(hybrid_sim, hybrid_sim)

In [151]:
def recommend_courses(course_index, top_n=5):
    scores = list(enumerate(similarity_matrix[course_index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_courses = [i for i, score in scores[1:top_n+1]]
    return metadata.iloc[top_courses][['Course_Name', 'About']]

# Example: recommend 5 courses similar to course 10
recommend_courses(10, top_n=5)

,Course_Name,About
116,App Marketing,This course will help you organize a strategy ...
101,Rapid Prototyping,Learning to prototype will save you time and m...
163,UX Design for Mobile Developers,This is a design course made for developers. Y...
148,Building High Conversion Web Forms,Increase conversions with best practices for f...
22,Data Product Manager,Hone specialized skills in Data Product Manage...


In [152]:
def recommend_by_text(query, top_n=5):
    # TF-IDF part
    query_tfidf = vectorizer.transform([query])
    tfidf_sim = cosine_similarity(query_tfidf, tfidf_matrix_norm)

    # spaCy part
    query_vec = nlp(query).vector.reshape(1, -1)
    spacy_sim = cosine_similarity(query_vec, spacy_matrix_norm)

    # Hybrid
    alpha, beta = 0.6, 0.4
    hybrid_scores = alpha * tfidf_sim + beta * spacy_sim

    # Get top matches
    top_idx = hybrid_scores.argsort()[0][-top_n:][::-1]
    return metadata.iloc[top_idx][['Course_Name', 'clean_text']]

# Example usage
recommend_by_text("I want to learn about Deep Learning", top_n=5)

,Course_Name,clean_text
40,Intro to Machine Learning with PyTorch,build solid foundation supervised unsupervised...
43,Intro to Machine Learning with TensorFlow,build solid foundation supervised unsupervised...
97,Intro to Deep Learning with PyTorch,learn basic deep learning implement deep neura...
8,Deep Learning,deep learning drive advance artificial intelli...
12,Machine Learning Engineer,machine learn engineer apply predictive model ...


In [153]:
# Recommending By Course Name

def recommend_by_course_name(course_name, top_n=5):
    # Find the index of the course with this name
    try:
        course_index = metadata[metadata['Course_Name'] == course_name].index[0]
    except IndexError:
        return f"Course '{course_name}' not found in metadata."

    # Get similarity scores for that course
    scores = list(enumerate(hybrid_sim[course_index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Take top N excluding the course itself
    top_courses = [i for i, score in scores[1:top_n+1]]
    
    return metadata.iloc[top_courses][['Course_Name', 'clean_text']]

In [154]:
recommend_by_course_name("Front-End Interview Prep", top_n=5)

,Course_Name,clean_text
259,Full-Stack Interview Prep,answer common stack web security interview que...
261,iOS Interview Prep,answer ios mobile development interview questi...
262,VR Interview Prep,learn tackle interview question technical role...
256,Android Interview Prep,learn respond common android mobile developmen...
105,Product Manager Interview Preparation,ace product management interview understand an...


In [166]:
import numpy as np

def assign_category(text):
    text = text.lower()
    if any(word in text for word in ["python", "java", "c++", "programming", "coding"]):
        return "Programming"
    elif any(word in text for word in ["data science", "pandas", "numpy", "analysis", "statistics"]):
        return "Data Science"
    elif any(word in text for word in ["machine learning", "ml", "supervised", "unsupervised"]):
        return "Machine Learning"
    elif any(word in text for word in ["deep learning", "neural network", "tensorflow", "pytorch"]):
        return "Deep Learning"
    elif any(word in text for word in ["ai", "artificial intelligence"]):
        return "Artificial Intelligence"
    elif any(word in text for word in ["math", "linear algebra", "probability"]):
        return "Math & Statistics"
    else:
        return "Other"

metadata["Category"] = metadata["clean_text"].apply(assign_category)
print(metadata[["Course_Name", "Category"]].head(10))

                   Course_Name                 Category
0                Data Engineer  Artificial Intelligence
1               Data Scientist         Machine Learning
2                 Data Analyst              Programming
3                          C++              Programming
4              Product Manager                    Other
5           Business Analytics  Artificial Intelligence
6  Introduction to Programming              Programming
7            Digital Marketing  Artificial Intelligence
8                Deep Learning            Deep Learning
9         Blockchain Developer  Artificial Intelligence


In [167]:
import random

students = range(1, 51)  # e.g., 50 students
interactions = []

for student in students:
    # choose 1–2 categories of interest
    categories = random.sample(metadata["Category"].unique().tolist(), k=2)

    for cat in categories:
        # pick 3 random courses from that category
        cat_courses = metadata[metadata["Category"] == cat]["Course_Name"]
        chosen_courses = cat_courses.sample(min(3, len(cat_courses)))
        
        for course in chosen_courses:
            interactions.append([student, course])

interactions_df = pd.DataFrame(interactions, columns=["Student_ID", "Course_Name"])
print(interactions_df.head())


   Student_ID                                    Course_Name
0           1                                  Intro to AJAX
1           1                    Gradle for Android and Java
2           1            Developing Android Apps with Kotlin
3           1                                          Spark
4           1  Machine Learning Engineer for Microsoft Azure


In [169]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_courses_by_name(course_name, df, k=5):
    # Find the course index
    idx = df[df["Course_Name"] == course_name].index[0]
    
    # Compute similarity of this course with all others
    cosine_similarities = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    
    # Get top k+1 because the course itself will be the most similar
    similar_indices = cosine_similarities.argsort()[::-1][1:k+1]
    
    # Return course names
    return df.iloc[similar_indices]["Course_Name"].tolist()


In [171]:
# Step 1: Build ground truth (pseudo interactions)
from sklearn.metrics.pairwise import cosine_similarity

# tfidf_matrix is from your clean_text
similarity = cosine_similarity(tfidf_matrix)

ground_truth = {}
for idx, course in enumerate(df["Course_Name"]):
    # take top 5 most similar courses (excluding itself)
    top_indices = similarity[idx].argsort()[::-1][1:6]
    ground_truth[course] = set(df["Course_Name"].iloc[top_indices])

# Step 2: Evaluate recommender
def evaluate_recommender(df, ground_truth, k=5):
    precisions = []
    for course in df["Course_Name"]:
        recommended = recommend_courses_by_name(course, df, k=k)
        relevant = ground_truth[course]
        intersection = set(recommended).intersection(relevant)
        precisions.append(len(intersection) / k)
    return sum(precisions) / len(precisions)

p_at_5 = evaluate_recommender(df, ground_truth, k=5)
print("Precision@5:", p_at_5)


Precision@5: 0.9931558935361217
